# Ex04 Enrichment and transformations

In [28]:
import pandas as pd
import numpy as np

## read json

In [29]:
data = pd.read_json('data/auto.json', orient = 'records')
data

,CarNumber,Refund,Fines,Make,Model
0,Y163O8161RUS,2,3200.00,Ford,Focus
1,E432XX77RUS,1,6500.00,Toyota,Camry
2,7184TT36RUS,1,2100.00,Ford,Focus
3,X582HE161RUS,2,2000.00,Ford,Focus
4,92918M178RUS,1,5700.00,Ford,Focus
...,...,...,...,...,...
720,Y163O8161RUS,2,1600.00,Ford,Focus
721,M0309X197RUS,1,22300.00,Ford,Focus
722,O673E8197RUS,2,600.00,Ford,Focus
723,8610T8154RUS,1,2000.00,Ford,Focus


In [30]:
data.dtypes

CarNumber     object
Refund         int64
Fines        float64
Make          object
Model         object
dtype: object

In [31]:
pd.options.display.float_format = '{:.2f}'.format

## enrich the dataframe using a sample from that dataframe

In [32]:
sample = data[['CarNumber','Make','Model']]
sample = sample.sample(n=200, random_state=21)
sample['Fines'] = np.random.choice(data.Fines, size = 200)
sample['Refund'] = np.random.choice(data.Refund, size = 200)
sample

,CarNumber,Make,Model,Fines,Refund
445,M0299X197RUS,Ford,Focus,11400.00,1
22,83298C154RUS,Ford,Focus,600.00,2
93,H957HY161RUS,Ford,Focus,6000.00,2
173,T941CC96RUS,Ford,Focus,4800.00,2
697,H966HY161RUS,Ford,Focus,46000.00,1
...,...,...,...,...,...
14,8182XX154RUS,Ford,Focus,1900.00,2
623,X796TH96RUS,Ford,Focus,75900.00,2
498,T011MY163RUS,Ford,Focus,500.00,1
536,T341CC96RUS,Volkswagen,Passat,9600.00,1


In [33]:
concat_rows = pd.concat([data,sample], axis = 0)
concat_rows

,CarNumber,Refund,Fines,Make,Model
0,Y163O8161RUS,2,3200.00,Ford,Focus
1,E432XX77RUS,1,6500.00,Toyota,Camry
2,7184TT36RUS,1,2100.00,Ford,Focus
3,X582HE161RUS,2,2000.00,Ford,Focus
4,92918M178RUS,1,5700.00,Ford,Focus
...,...,...,...,...,...
14,8182XX154RUS,2,1900.00,Ford,Focus
623,X796TH96RUS,2,75900.00,Ford,Focus
498,T011MY163RUS,1,500.00,Ford,Focus
536,T341CC96RUS,1,9600.00,Volkswagen,Passat


In [51]:
concat_rows.count()

CarNumber    925
Refund       925
Fines        925
Make         925
Model        925
dtype: int64

## enrich the dataframe concat_rows by a new column with the data generated

In [34]:
np.random.seed(21)
years = pd.Series(np.random.choice(range(1980,2019), size = 725), name = 'Year', dtype=int)
fines = pd.concat([concat_rows, years], axis = 1)
fines

,CarNumber,Refund,Fines,Make,Model,Year
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995
2,7184TT36RUS,1,2100.00,Ford,Focus,1984
3,X582HE161RUS,2,2000.00,Ford,Focus,2015
4,92918M178RUS,1,5700.00,Ford,Focus,2014
...,...,...,...,...,...,...
14,8182XX154RUS,2,1900.00,Ford,Focus,2017
623,X796TH96RUS,2,75900.00,Ford,Focus,2013
498,T011MY163RUS,1,500.00,Ford,Focus,2015
536,T341CC96RUS,1,9600.00,Volkswagen,Passat,1986


In [35]:
fines.count()

CarNumber    925
Refund       925
Fines        925
Make         925
Model        925
Year         925
dtype: int64

## enrich the dataframe with the data from another dataframe

In [36]:
surnames = pd.read_json('data/surname.json', orient='series')
random_surnames = surnames[0][1:].sample(n = fines.CarNumber.nunique(), random_state = 21, replace = True)
random_surnames.reset_index(drop = True)


0      RICHARDSON
1            ROSS
2          MORGAN
3          BAILEY
4           LOPEZ
          ...    
526      CAMPBELL
527          HALL
528         BAKER
529          DIAZ
530        MORGAN
Name: 0, Length: 531, dtype: object

In [37]:
surnames_df = pd.DataFrame(fines.CarNumber.unique())
surnames_df['SURNAME'] = np.array(random_surnames)

In [38]:
surnames_df.rename(columns = ({0 : 'CarNumber'}), inplace = True)
surnames_df

,CarNumber,SURNAME
0,Y163O8161RUS,RICHARDSON
1,E432XX77RUS,ROSS
2,7184TT36RUS,MORGAN
3,X582HE161RUS,BAILEY
4,92918M178RUS,LOPEZ
...,...,...
526,O136HO197RUS,CAMPBELL
527,O22097197RUS,HALL
528,M0309X197RUS,BAKER
529,O673E8197RUS,DIAZ


In [39]:
new_fines = pd.DataFrame([
    {'CarNumber' : 'Y200B9122RUS', 'Refund' : 2, 'Fines' : 5400.50, 'Make' : 'BMW', 'Model' : 'M5', 'Year' : 1990},
    {'CarNumber' : 'X200B9124RUS', 'Refund' : 1, 'Fines' : 2450.50, 'Make' : 'Audi', 'Model' : 'A4', 'Year' : 2000},
    {'CarNumber' : 'D900B9152RUS', 'Refund' : 2, 'Fines' : 9200.50, 'Make' : 'Ford', 'Model' : 'Focus', 'Year' : 2007},
    {'CarNumber' : 'K220B9322RUS', 'Refund' : 1, 'Fines' : 2790.50, 'Make' : 'BMW', 'Model' : 'M5', 'Year' : 1993},
    {'CarNumber' : 'L321B9144RUS', 'Refund' : 2, 'Fines' : 8100.50, 'Make' : 'Toyota', 'Model' : 'Camry', 'Year' : 2015}]
)
fines = pd.concat([fines, new_fines])

In [40]:
# delete last 20 samples
surnames_df = surnames_df[:-20]

In [41]:
surnames_df

,CarNumber,SURNAME
0,Y163O8161RUS,RICHARDSON
1,E432XX77RUS,ROSS
2,7184TT36RUS,MORGAN
3,X582HE161RUS,BAILEY
4,92918M178RUS,LOPEZ
...,...,...
506,T914CT197RUS,HERNANDEZ
507,E41977152RUS,BAKER
508,9464EX178RUS,MARTIN
509,O50197197RUS,WRIGHT


In [42]:
new_surnames = pd.DataFrame([{'CarNumber': 'O718FF163RUS', 'SURNAME' : 'BLACK'},
                              {'CarNumber': 'N925FF163RUS', 'SURNAME' : 'SMITH'},
                              {'CarNumber': 'M718FF197RUS', 'SURNAME' : 'MESSI'}])
surnames_df = pd.concat([surnames_df, new_surnames])

In [43]:
surnames_df

,CarNumber,SURNAME
0,Y163O8161RUS,RICHARDSON
1,E432XX77RUS,ROSS
2,7184TT36RUS,MORGAN
3,X582HE161RUS,BAILEY
4,92918M178RUS,LOPEZ
...,...,...
509,O50197197RUS,WRIGHT
510,7608EE777RUS,HILL
0,O718FF163RUS,BLACK
1,N925FF163RUS,SMITH


## merge

In [44]:
# CarNumber in fines and surnames_df
pd.merge(fines, surnames_df, on='CarNumber', how = 'inner')

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989,RICHARDSON
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995,ROSS
2,7184TT36RUS,1,2100.00,Ford,Focus,1984,MORGAN
3,X582HE161RUS,2,2000.00,Ford,Focus,2015,BAILEY
4,92918M178RUS,1,5700.00,Ford,Focus,2014,LOPEZ
...,...,...,...,...,...,...,...
894,8182XX154RUS,2,1900.00,Ford,Focus,2017,SMITH
895,X796TH96RUS,2,75900.00,Ford,Focus,2013,WATSON
896,T011MY163RUS,1,500.00,Ford,Focus,2015,SANDERS
897,T341CC96RUS,1,9600.00,Volkswagen,Passat,1986,PEREZ


In [45]:
# CarNumber in fines or surnames_df
pd.merge(fines, surnames_df, on='CarNumber', how = 'outer')

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,704687163RUS,2.00,1400.00,Ford,Focus,2014.00,ADAMS
1,704787163RUS,2.00,2800.00,Ford,Focus,2005.00,MORGAN
2,704987163RUS,2.00,8594.59,Ford,Focus,2014.00,MITCHELL
3,705287163RUS,2.00,2000.00,Ford,Focus,1990.00,GOMEZ
4,705387163RUS,2.00,700.00,Ford,Focus,2005.00,STEWART
...,...,...,...,...,...,...,...
928,Y973O8197RUS,2.00,8594.59,Ford,Focus,2005.00,YOUNG
929,Y973O8197RUS,1.00,34800.00,Ford,Focus,2013.00,YOUNG
930,Y973O8197RUS,1.00,69600.00,Ford,Focus,1989.00,YOUNG
931,Y973O8197RUS,1.00,100.00,Ford,Focus,2013.00,YOUNG


In [46]:
# CarNumber in fines
pd.merge(fines, surnames_df, on='CarNumber', how = 'left')

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989,RICHARDSON
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995,ROSS
2,7184TT36RUS,1,2100.00,Ford,Focus,1984,MORGAN
3,X582HE161RUS,2,2000.00,Ford,Focus,2015,BAILEY
4,92918M178RUS,1,5700.00,Ford,Focus,2014,LOPEZ
...,...,...,...,...,...,...,...
925,Y200B9122RUS,2,5400.50,BMW,M5,1990,NaN
926,X200B9124RUS,1,2450.50,Audi,A4,2000,NaN
927,D900B9152RUS,2,9200.50,Ford,Focus,2007,NaN
928,K220B9322RUS,1,2790.50,BMW,M5,1993,NaN


In [47]:
# CarNumber in surnames_df
pd.merge(fines, surnames_df, on='CarNumber', how = 'right')

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2.00,3200.00,Ford,Focus,1989.00,RICHARDSON
1,Y163O8161RUS,2.00,1600.00,Ford,Focus,1999.00,RICHARDSON
2,E432XX77RUS,1.00,6500.00,Toyota,Camry,1995.00,ROSS
3,E432XX77RUS,2.00,13000.00,Toyota,Camry,1992.00,ROSS
4,7184TT36RUS,1.00,2100.00,Ford,Focus,1984.00,MORGAN
...,...,...,...,...,...,...,...
897,7608EE777RUS,1.00,4000.00,Skoda,Octavia,2013.00,HILL
898,7608EE777RUS,2.00,8000.00,Skoda,Octavia,2013.00,HILL
899,O718FF163RUS,NaN,NaN,NaN,NaN,NaN,BLACK
900,N925FF163RUS,NaN,NaN,NaN,NaN,NaN,SMITH


## pivot_table

In [48]:
pd.pivot_table(fines, values = 'Fines', index = ['Make', 'Model'], columns = 'Year', aggfunc = 'sum')

Year                   1980      1981     1982      1983      1984      1985  \
Make       Model                                                               
Audi                    NaN       NaN      NaN       NaN       NaN       NaN   
           A4           NaN       NaN      NaN       NaN       NaN       NaN   
BMW                     NaN       NaN      NaN       NaN       NaN       NaN   
           M5           NaN       NaN      NaN       NaN       NaN       NaN   
Ford       Focus   78100.00 246594.59 62800.00 192394.59 104600.00 259394.59   
           Mondeo       NaN       NaN 46200.00       NaN       NaN       NaN   
Skoda      Octavia  8594.59   1900.00  8894.59       NaN    300.00  24394.59   
Toyota     Camry   12000.00       NaN      NaN       NaN   1400.00       NaN   
           Corolla      NaN   6800.00      NaN  12800.00       NaN       NaN   
Volkswagen          1300.00   7900.00      NaN       NaN       NaN       NaN   
           Golf    23300.00   8594.59  5000.00    200.00       NaN 168000.00   
           Jetta        NaN   1000.00      NaN       NaN       NaN   9000.00   
           Passat    900.00   3000.00      NaN   1100.00   8594.59       NaN   
           Touareg      NaN       NaN      NaN       NaN       NaN       NaN   
Volvo                   NaN       NaN      NaN       NaN       NaN       NaN   

Year                   1986      1987     1988      1989  ...      2009  \
Make       Model                                          ...             
Audi                    NaN       NaN      NaN       NaN  ...       NaN   
           A4           NaN       NaN      NaN       NaN  ...       NaN   
BMW                     NaN       NaN      NaN       NaN  ...       NaN   
           M5           NaN       NaN      NaN       NaN  ...       NaN   
Ford       Focus   76094.59 212294.59 51078.35 204589.17  ... 158694.59   
           Mondeo       NaN       NaN      NaN       NaN  ...       NaN   
Skoda      Octavia      NaN   2000.00  5100.00   8594.59  ...       NaN   
Toyota     Camry   19800.00       NaN      NaN   4600.00  ...       NaN   
           Corolla      NaN  84000.00      NaN   9300.00  ...   8594.59   
Volkswagen          7400.00       NaN      NaN       NaN  ...       NaN   
           Golf         NaN       NaN      NaN   5400.00  ...       NaN   
           Jetta        NaN       NaN 46000.00       NaN  ...       NaN   
           Passat  28600.00   2000.00  8594.59       NaN  ...       NaN   
           Touareg      NaN       NaN      NaN       NaN  ...   5800.00   
Volvo                   NaN       NaN      NaN       NaN  ...   6800.00   

Year                    2010      2011      2012      2013     2014      2015  \
Make       Model                                                                
Audi                     NaN       NaN       NaN       NaN      NaN       NaN   
           A4            NaN       NaN       NaN       NaN      NaN       NaN   
BMW                      NaN   8594.59       NaN       NaN      NaN       NaN   
           M5            NaN       NaN       NaN       NaN      NaN       NaN   
Ford       Focus   128094.59 147689.17 297983.76 392089.17 75783.76 230000.00   
           Mondeo        NaN       NaN       NaN  41100.00      NaN       NaN   
Skoda      Octavia   2900.00  19000.00   1700.00  19800.00      NaN  16694.59   
Toyota     Camry    24400.00       NaN   7500.00       NaN      NaN   8100.50   
           Corolla   6000.00       NaN       NaN       NaN      NaN       NaN   
Volkswagen               NaN       NaN       NaN       NaN      NaN       NaN   
           Golf          NaN       NaN       NaN       NaN 17900.00       NaN   
           Jetta         NaN       NaN       NaN       NaN      NaN       NaN   
           Passat   14900.00       NaN       NaN   4600.00      NaN       NaN   
           Touareg       NaN       NaN       NaN       NaN      NaN       NaN   
Volvo                    NaN       NaN       NaN       NaN      NaN

## save_csv

In [49]:
fines.to_csv('data/fines.csv', index = False)

In [50]:
surnames_df.to_csv('data/surnames.csv', index = False)